In [1]:
from typing import Tuple, List, Dict, Any, Sequence
import polars as pl
import polars.selectors as cs
import pandas as pd
from pprint import pprint, pformat
import sys
from pathlib import Path
from tqdm import tqdm, trange
import psutil
import re


# 상대 경로 사용
PROJECT_ROOT = Path.cwd().parent
DATA_DIR = PROJECT_ROOT / 'data'

# 맨 앞에 추가
if str(PROJECT_ROOT) in sys.path:
    sys.path.remove(str(PROJECT_ROOT))
sys.path.insert(0, str(PROJECT_ROOT))

# Python 내장 code 모듈 캐시만 임시 제거
if 'code' in sys.modules:
    del sys.modules['code']

# 이제 import
from code.utils import process_lazyframe_in_chunks
from code.loading import DataLoader
from code.preprocess import TextPreprocessor, create_udi_preprocessor, create_company_preprocessor, create_generic_preprocessor
from code.preprocess.preprocess import get_pattern_cols, \
    get_unique_by_cols_safe, get_unique, \
    analyze_null_values, replace_pattern_with_null, overview_col

## 원본 데이터 불러오기

In [2]:
# maude 데이터 불러오기
loader1 = DataLoader(
    start=2020,
    end=2025,
    output_file = DATA_DIR / 'maude.parquet',
    max_workers=4
)

adapter = 'polars'
polars_kwargs = {
    'use_statistics': True,
    'parallel': 'auto',
    'low_memory': False,
    'rechunk': False,
    'cache': True,
}
maude_lf = loader1.load(adapter=adapter, **polars_kwargs)
maude_lf


📖 /home/dataiku/eric/Sparta/Project3/data/maude.parquet 로딩 중... (adapter=polars)


In [3]:
# udi 데이터 불러오기
udi_loader = DataLoader(
    name='udi',
    output_file=DATA_DIR/'udi.parquet',
)

udi_lf = udi_loader.load(adapter, **polars_kwargs)
udi_lf


📖 /home/dataiku/eric/Sparta/Project3/data/udi.parquet 로딩 중... (adapter=polars)


## UDI Dataset 전처리

In [4]:
IDENTIFIER_PATTERNS = [
    r"^device_\d+_brand_name$",
    r"identifiers_\d+_id", 
    r"identifiers_\d+_issuing_agency", 
    r"identifiers_\d+_package_discontinue_date", 
    r"identifiers_\d+_package_status", 
    r"identifiers_\d+_package_type", 
    r"identifiers_\d+_quantity_per_package", 
    r"identifiers_\d+_type", 
    r"identifiers_\d+_unit_of_use_id"
]
UDI_DI_PATTERNS = [r'^identifiers_\d+_id$']
TYPE_PATTERNS = [
    r'identifiers_\d+_type'
]

CUSTOMER_PATTERNS = [r'^customer']
DEVICE_SIZE_PATTERNS = [r'^device_sizes']
STORAGE_PATTERNS = [r'^storage']

### Drop 필요없는 열

In [5]:
drop_patterns = CUSTOMER_PATTERNS + DEVICE_SIZE_PATTERNS + STORAGE_PATTERNS

regex = "|".join(drop_patterns)

udi_lf = udi_lf.select(
    ~cs.matches(regex)
)

# udi_lf.collect_schema().names()

### Primary 추출

In [6]:
udi_di_cols = get_pattern_cols(udi_lf, UDI_DI_PATTERNS)
identifiers_cols = get_pattern_cols(udi_lf, IDENTIFIER_PATTERNS)
type_cols = get_pattern_cols(udi_lf, TYPE_PATTERNS)

In [7]:
# type-udi_di 쌍 만들기 (인덱스로 매칭)
def extract_index(col_name):
    match = re.search(r'identifiers_(\d+)_', col_name)
    return int(match.group(1)) if match else None

type_id_pairs = []
for type_col in type_cols:
    idx = extract_index(type_col)
    udi_di_col = f'identifiers_{idx}_id'
    if udi_di_col in udi_di_cols:
        type_id_pairs.append((type_col, udi_di_col))

len(type_id_pairs)

132

In [8]:
primary_udi_unique = set()

for type_col, id_col in tqdm(type_id_pairs, desc="Processing columns", unit="col"):
    try:
        count = (
            udi_lf
            .filter(pl.col(type_col).eq("Primary"))
            .select(pl.len())
            .collect()
            .item()
        )
        
        if count > 0:
            ids = (
                udi_lf
                .filter(pl.col(type_col).eq("Primary"))
                .select(pl.col(id_col))
                .unique()
                .collect()
                .to_series()
                .drop_nulls()
                .to_list()
            )
            primary_udi_unique.update(ids)
            
    except Exception as e:
        tqdm.write(f"Error processing {type_col}: {e}")
        continue

print(f"\n{'='*50}")
print(f"UDI 데이터의 고유 Primary udi 개수: {len(primary_udi_unique):,}")
print(f"{'='*50}")

Processing columns: 100%|██████████| 132/132 [00:01<00:00, 79.21col/s] 


UDI 데이터의 고유 Primary udi 개수: 4,903,764


In [9]:
udi_step1_path = DATA_DIR / 'silver' / 'udi_primary.parquet'
udi_step2_path = DATA_DIR / 'silver' / 'udi_clean.parquet'

In [10]:
# # Primary인 id를 추출
# def primary_transform(lf: pl.LazyFrame):
#     return lf.with_columns(
#         pl.coalesce([
#             pl.when(pl.col(type_col).eq("Primary"))
#             .then(pl.col(id_col))
#             for type_col, id_col in type_id_pairs
#         ]).alias('primary_udi_di')
#     )

# process_lazyframe_in_chunks(
#     udi_lf, 
#     primary_transform, 
#     udi_step1_path, 
#     10_000,
#     desc="Primary extraction"
# )

In [11]:
# # udi 데이터 불러오기
# udi_loader = DataLoader(
#     name='udi',
#     output_file=udi_step1_path,
# )

# primary_udi_lf = udi_loader.load(adapter, **polars_kwargs)

In [12]:
# # 회사 이름 정제
# preprocessor1 = create_company_preprocessor()

# preprocessor1.apply_to_lazyframe(
#     primary_udi_lf,
#     ['company_name', 'brand_name'],
#     udi_step2_path,
#     10_000,
# )

# del preprocessor1, primary_udi_lf
# udi_step1_path.unlink(missing_ok=True)

In [13]:
# udi 데이터 불러오기
udi_loader = DataLoader(
    name='udi',
    output_file=udi_step2_path,
)

cleaned_udi_lf = udi_loader.load(adapter, **polars_kwargs)


📖 /home/dataiku/eric/Sparta/Project3/data/silver/udi_clean.parquet 로딩 중... (adapter=polars)


In [14]:
# maude_lf.filter(
#     pl.col('device_0_udi_di').is_not_null()
# ).group_by(
#     ['device_0_manufacturer_d_name', 'device_0_brand_name', 'device_0_model_number', 'device_0_catalog_number']
# ).agg(
#     pl.col('device_0_udi_di').n_unique().alias('udi_nunique'),
#     pl.col('device_0_udi_di').unique().alias('udi_unique'),
#     pl.col('device_0_udi_di').mode().alias('udi_mode')
# ).sort('udi_nunique', descending=True).head(10).collect().to_pandas()

## MAUDE 1차 전처리

In [15]:
# 기본 변수
BASE_COLS = [
    'mdr_report_key', 'report_number', 'adverse_event_flag', 'product_problem_flag', 
    'date_of_event', 'date_received', 'device_date_of_manufacturer', 'event_type',
    'previous_use_code', 'single_use_flag', 'report_source_code',
    'reprocessed_and_reused_flag', 'report_to_fda', 'event_location', 
    'manufacturer_link_flag', 'manufacturer_g1_name', 'manufacturer_g1_postal_code',
    'pma_pmn_number'
]

DEVICE_COLS = [
    "device_0_manufacturer_d_name",
    "device_0_manufacturer_d_postal_code",
    "device_0_brand_name",
    "device_0_catalog_number",
    "device_0_model_number",
    "device_0_udi_di",
    "device_0_lot_number",
    "device_0_udi_public",
    "device_0_device_report_product_code",
    "device_0_device_age_text",
    "device_0_device_operator",
    "device_0_openfda_device_class",
    "device_0_openfda_device_name",
]



PATIENT_COLS = [
    "patient_0_patient_sequence_number",
    "patient_0_patient_age",
    "patient_0_patient_sex",
    "patient_0_patient_weight",
    "patient_0_patient_race",
    "patient_0_patient_problems",
    "patient_0_sequence_number_outcome",
    "patient_0_sequence_number_treatment",
]



MDR_TEXT_PATTERNS = [
    r"^mdr_text_.*_text$",
    r"^mdr_text_.*_text_type_code$",
]

MDR_COLS = get_pattern_cols(maude_lf, MDR_TEXT_PATTERNS)
TOTAL_COLS = BASE_COLS + DEVICE_COLS + PATIENT_COLS + MDR_COLS

### Drop 필요 없는 열

In [16]:
maude_lf = maude_lf.select(TOTAL_COLS)

# maude_lf.collect_schema().names()

In [17]:
maude_step1_path = DATA_DIR / 'silver' / 'clean_step1.parquet'
maude_step2_path = DATA_DIR / 'silver' / 'clean_step2.parquet'
maude_step3_path = DATA_DIR / 'silver' / 'clean_step3.parquet'

In [18]:
# # 1단계
# preprocessor1 = create_udi_preprocessor()
# preprocessor1.apply_to_lazyframe(
#     maude_lf, 'device_0_udi_di', maude_step1_path, chunk_size=10_000
# )
# del preprocessor1  # 명시적 삭제

In [19]:
# # 2단계
# maude_lf2 = pl.scan_parquet(maude_step1_path)
# preprocessor2 = create_company_preprocessor()
# preprocessor2.apply_to_lazyframe(
#     maude_lf2, ['device_0_manufacturer_d_name', 'manufacturer_g1_name', 'device_0_brand_name'], 
#     maude_step2_path, chunk_size=10_000
# )
# del maude_lf2, preprocessor2

In [20]:
# # 중간 파일 정리
# maude_step1_path.unlink(missing_ok=True)

# # 3단계
# maude_lf3 = pl.scan_parquet(maude_step2_path)
# preprocessor3 = create_generic_preprocessor()
# preprocessor3.apply_to_lazyframe(
#     maude_lf3, ['device_0_model_number', 'device_0_catalog_number', 'device_0_lot_number'], 
#     maude_step3_path, chunk_size=10_000
# )
# del maude_lf3, preprocessor3

In [21]:
maude_step2_path.unlink(missing_ok=True)

# maude 데이터 불러오기
loader3 = DataLoader(
    start=2020,
    end=2025,
    output_file = maude_step3_path,
)

adapter = 'polars'
polars_kwargs = {
    'use_statistics': True,
    'parallel': 'auto',
    'low_memory': False,
    'rechunk': False,
    'cache': True,
}
cleaned_maude_lf = loader3.load(adapter=adapter, **polars_kwargs)
cleaned_maude_lf


📖 /home/dataiku/eric/Sparta/Project3/data/silver/clean_step3.parquet 로딩 중... (adapter=polars)


## 작업에 필요한 컬럼

In [22]:
rename_udi_lf = cleaned_udi_lf.rename({
    'company_name': 'manufacturer',
    'brand_name': 'brand',
    'version_or_model_number': 'model_number',
    'primary_udi_di': 'udi_di',
})

rename_maude_lf = cleaned_maude_lf.rename({
    'device_0_manufacturer_d_name': 'manufacturer',
    'device_0_brand_name': 'brand',
    'device_0_model_number': 'model_number',
    'device_0_catalog_number': 'catalog_number',
    'device_0_lot_number': 'lot_number',
    'device_0_udi_di': 'udi_di',
})

In [23]:
target_cols = [
    'manufacturer',
    'brand',
    'model_number',
    'catalog_number'
]

join_col = 'udi_di'

common_cols = target_cols + [join_col]

maude_cols = common_cols + [
    'mdr_report_key',
]

udi_cols = common_cols + udi_di_cols

In [24]:
udi_necessary_lf = rename_udi_lf.select(pl.col(udi_cols))
maude_necessary_lf = rename_maude_lf.select(pl.col(maude_cols))

## 고유값 추출

In [25]:
cols_group = {
    'udi': udi_di_cols,
}

# udi 데이터셋의 udi_di 고유값
udi_udi_unique = get_unique_by_cols_safe(
    udi_necessary_lf, 
    cols_group,
    memory_safety_ratio=0.3,
    calibration_factor = 1
)['udi']

문자열 길이 통계를 실제 데이터로부터 추정 중...
  문자열 길이 통계:
    - 평균(mean): 13.9자
    - 중앙값(median): 14.0자
    - 75%ile: 14.0자
    - 90%ile: 14.0자
    - 최댓값: 23자
  → 사용할 크기(p75): 14.0자

=== 메모리 기반 자동 임계값 계산 ===
사용 가능한 메모리: 94.81 GB
안전 사용 메모리 (30%): 28.44 GB
예상 바이트/고유값: 106 bytes
  - 문자열 데이터: 28 bytes
  - str 오버헤드: 50 bytes
  - set 오버헤드: 28 bytes
계산된 최대 고유값: 288,106,325개



Extracting unique values:   0%|          | 0/1 [00:00<?, ?it/s]

udi: 6,336,862개의 고유값 (예상 메모리: 640.6 MB)


Extracting unique values: 100%|██████████| 1/1 [00:05<00:00,  5.56s/it]

  ✓ udi 추출 완료 (실제 메모리: 636.11 MB)
    → 예상치 정확도: 100.7% (예상/실제 비율)


=== 추출 요약 ===
성공: 1/1
실패/스킵: 0/1


In [26]:
maude_udi_unique = get_unique(maude_necessary_lf, ['udi_di'])
angry_udi_unique = maude_udi_unique - udi_udi_unique
survive_udi_unique = maude_udi_unique & udi_udi_unique

print(f'UDI 데이터의 고유 udi 개수: {len(udi_udi_unique)}개')
print(f'MAUDE 데이터의 고유 udi 개수: {len(maude_udi_unique)}개')
print(f'UDI 데이터에 없는 MAUDE 데이터의 고유 udi 개수: {len(angry_udi_unique)}개')
print(f'UDI 데이터에 있는 MAUDE 데이터의 고유 udi 개수: {len(survive_udi_unique)}개')

UDI 데이터의 고유 udi 개수: 6336861개
MAUDE 데이터의 고유 udi 개수: 142436개
UDI 데이터에 없는 MAUDE 데이터의 고유 udi 개수: 30780개
UDI 데이터에 있는 MAUDE 데이터의 고유 udi 개수: 111656개


In [27]:
maude_primary_udi_unique = survive_udi_unique & primary_udi_unique
maude_secondary_udi_unique = survive_udi_unique - primary_udi_unique

print(f'Primary UDI인 MAUDE 데이터의 고유 udi 개수: {len(maude_primary_udi_unique)}개')
print(f'Primary UDI가 아닌 MAUDE 데이터의 고유 udi 개수: {len(maude_secondary_udi_unique)}개')

Primary UDI인 MAUDE 데이터의 고유 udi 개수: 105199개
Primary UDI가 아닌 MAUDE 데이터의 고유 udi 개수: 6457개


In [28]:
class UniqueUDIDI:
    def __init__(
        self,
        udi_udi_unique: set = None,
        maude_udi_unique: set = None,
        angry_udi_unique: set = None,
        survive_udi_unique: set = None,
        primary_udi_unique: set = None,
        maude_primary_udi_unique: set = None,
        maude_secondary_udi_unique: set = None
    ):
        self.udi = udi_udi_unique
        self.maude = maude_udi_unique
        self.angry = angry_udi_unique
        self.survive = survive_udi_unique
        self.primary = primary_udi_unique
        self.maude_primary = maude_primary_udi_unique
        self.maude_secondary = maude_secondary_udi_unique

    def print_stats(self):
        print(f'UDI 데이터의 고유 udi 개수: {len(self.udi)}개')
        print(f'UDI 데이터의 고유 Primary udi 개수: {len(self.primary)}개')
        print(f'MAUDE 데이터의 고유 udi 개수: {len(self.maude)}개')
        print(f'UDI 데이터에 있는 MAUDE 데이터의 고유 udi 개수: {len(self.survive)}개')
        print(f'Primary UDI인 MAUDE 데이터의 고유 udi 개수: {len(self.maude_primary)}개')
        print(f'Primary UDI가 아닌 MAUDE 데이터의 고유 udi 개수: {len(self.maude_secondary)}개')
        print(f'UDI 데이터에 없는 MAUDE 데이터의 고유 udi 개수: {len(self.angry)}개')

unique_udi_di = UniqueUDIDI(udi_udi_unique, maude_udi_unique, angry_udi_unique, survive_udi_unique, primary_udi_unique, maude_primary_udi_unique, maude_secondary_udi_unique)

unique_udi_di.print_stats()

UDI 데이터의 고유 udi 개수: 6336861개
UDI 데이터의 고유 Primary udi 개수: 4903764개
MAUDE 데이터의 고유 udi 개수: 142436개
UDI 데이터에 있는 MAUDE 데이터의 고유 udi 개수: 111656개
Primary UDI인 MAUDE 데이터의 고유 udi 개수: 105199개
Primary UDI가 아닌 MAUDE 데이터의 고유 udi 개수: 6457개
UDI 데이터에 없는 MAUDE 데이터의 고유 udi 개수: 30780개


## UDI 데이터셋에 Primary 컬럼 생성

In [29]:
# MAUDE UDI를 LazyFrame으로
maude_udi_lf = pl.LazyFrame({
    'udi_di': list(unique_udi_di.maude_secondary)
})

In [30]:
# unpivot 후 즉시 필터링 (메모리 증가 최소화)
udi_mapping_lf = (
    udi_necessary_lf
    .with_row_index('row_idx')
    .unpivot(
        index=['row_idx'] + common_cols,
        on=udi_di_cols,
        variable_name='matched_col',
        value_name='match_udi_di'
    )
    .filter(
        pl.col('match_udi_di').is_not_null() &  # null 제거
        pl.col('match_udi_di').is_in(unique_udi_di.maude_secondary)  # 매칭되는 것만
    )
    .unique(subset=['match_udi_di', 'row_idx'])
    .select(['match_udi_di', 'row_idx'] + common_cols)
)

In [31]:
udi_mapping_df = udi_mapping_lf.collect().to_pandas()

In [32]:
# 통계만 먼저 확인
print(f"Total matches: {len(udi_mapping_df):,}")
print(f"Unique UDIs: {udi_mapping_df['match_udi_di'].nunique():,}")

Total matches: 6,709
Unique UDIs: 6,457


In [33]:
udi_mapping_df[udi_mapping_df['match_udi_di'] == '00021292007706']

,match_udi_di,row_idx,manufacturer,brand,model_number,catalog_number,udi_di
825,00021292007706,4081763,TRIVIDIA HEALTH,TRUE METRIX GO,TRUE METRIX GO Self Monitoring Blood Glucose ...,RF4001-01BK,00311917178691
1634,00021292007706,958701,TRIVIDIA HEALTH,TRUE METRIX GO,TRUE METRIX GO Self Monitoring Blood Glucose ...,RF4066-01,00011822002783
2253,00021292007706,3978446,TRIVIDIA HEALTH,TRUE METRIX GO,TRUE METRIX GO Self Monitoring Blood Glucose M...,RF4H01-01BK,00021292006112
4245,00021292007706,1231688,TRIVIDIA HEALTH,TRUE METRIX GO,TRUE METRIX GO Self Monitoring Blood Glucose M...,RF4023-01,00021292009335
4448,00021292007706,1719543,TRIVIDIA HEALTH,TRUE METRIX GO,TRUE METRIX GO Self Monitoring Blood Glucose M...,RF4209-01,00036800234567
4886,00021292007706,3344311,TRIVIDIA HEALTH,TRUE METRIX GO,TRUE METRIX GO Self-Monitoring Blood Glucose M...,RF4007-01,00050428560402
5125,00021292007706,1146676,TRIVIDIA HEALTH,TRUE METRIX GO,TRUE METRIX GO Self Monitoring Blood Glucose M...,RF4019-01,00708820721098
5266,00021292007706,2292496,TRIVIDIA HEALTH,TRUE METRIX GO,TRUE METRIX GO Self Monitoring Blood Glucose M...,RF4H01-40,00021292009793
5371,00021292007706,3997979,TRIVIDIA HEALTH,TRUE METRIX GO,HEB TRUE METRIX GO Self Monitoring Blood Gluco...,RF4031-01,00041220136131


In [34]:
maude_necessary_lf.filter(
    pl.col('udi_di').eq('00021292007706')
).head(15).collect().to_pandas().transpose()

,0,1,2,3,4,5,6,7,8,9,10
manufacturer,TRIVIDIA HEALTH,TRIVIDIA HEALTH,TRIVIDIA HEALTH,TRIVIDIA HEALTH,TRIVIDIA HEALTH,TRIVIDIA HEALTH,TRIVIDIA HEALTH,TRIVIDIA HEALTH,TRIVIDIA HEALTH,TRIVIDIA HEALTH,TRIVIDIA HEALTH
brand,TRUE METRIX GO,TRUE METRIX GO,TRUE METRIX GO,TRUE METRIX GO,TRUE METRIX GO,TRUE METRIX GO,TRUE METRIX GO,TRUE METRIX GO,TRUE METRIX GO,TRUE METRIX GO,TRUE METRIX GO
model_number,KIT GO CVS TMX METERMG/DL,KIT GO TVH TMX METERMG/DL,KIT GO CVS TMX METERMG/DL,KIT GO CVS TMX METERMG/DL,KIT GO WGN TMX METERMG/DL #910395,KIT GO CVS TMX METERMG/DL,KIT GO TVH TMX METERMG/DL,KIT GO TVH TMX METERMG/DL,KIT GO CVS TMX METERMG/DL,KIT GO CVS TMX METERMG/DL,KIT GO WGN TMX METERMG/DL #910395
catalog_number,RF4H01-01,None,RF4H01-01,None,RF4H01-01,RF4H01-01,None,RF4H01-01,RF4H01-01,RF4H01-01,None
udi_di,00021292007706,00021292007706,00021292007706,00021292007706,00021292007706,00021292007706,00021292007706,00021292007706,00021292007706,00021292007706,00021292007706
mdr_report_key,9965105,20041268,11761582,18722016,10334975,15319830,22201561,11333095,15455793,11085059,22742611


# 전처리 함수 설계

In [35]:
# 일치 점수 매기는 함수
# 1. 매핑한 행만 불러오기
# 2. company_name, brand_name, version_or_mode_number, catalog_number에 대하여 
# 2-1. 전부 대문자로
# 2-2. 일치 점수 매기기
# 2-3. 가장 점수가 높은 것의 primary_udi_di를 할당
# 2-4. 필요한 정보 가져오기

# udi를 깔끔하게 만듦
# primary: 그대로 primary_lf와 매칭해서 필요 info 가져옴
# secondary: primary_lf 중 udi 매칭한 lf에서 점수 높은 info 가져옴
# null: primary_lf에서 점수 높은 info 가져옴

### primary와 매칭해서 가져오는 함수

In [44]:
def extract_from_match(
    src_lf: pl.LazyFrame, desc_lf: pl.LazyFrame, 
    on: str | Sequence[str], 
    target_cols: str | Sequence[str]
):
    if isinstance(on, str):
        on = [on]
    
    if isinstance(target_cols, str):
        target_cols = [target_cols]
    
    udi_subset = src_lf.select(pl.col(on + target_cols))
    
    result = desc_lf.join(
        udi_subset,
        on=on,
        how='left'
    )
    
    for col in target_cols:
        result = result.with_columns(
            pl.coalesce([f'{col}_right', col]).alias(col)
        ).drop(f'{col}_right')
    
    return result

maude_match_lf = extract_from_match(udi_necessary_lf, rename_maude_lf, join_col, target_cols)

# cleaned_maude_lf.head(10).collect().to_pandas()
maude_match_lf.head(10).collect().to_pandas()

,mdr_report_key,report_number,adverse_event_flag,product_problem_flag,date_of_event,date_received,device_date_of_manufacturer,event_type,previous_use_code,single_use_flag,...,mdr_text_78_text,mdr_text_78_text_type_code,mdr_text_79_text,mdr_text_79_text_type_code,mdr_text_7_text,mdr_text_7_text_type_code,mdr_text_8_text,mdr_text_8_text_type_code,mdr_text_9_text,mdr_text_9_text_type_code
0,10437180,2032227-2020-166684,N,Y,20200815,20200821,20190405,Malfunction,U,N,...,None,None,None,None,None,None,None,None,None,None
1,10441163,2032227-2020-167028,N,Y,20200817,20200824,20190713,Malfunction,U,N,...,None,None,None,None,None,None,None,None,None,None
2,10452637,2032227-2020-168309,N,Y,20200820,20200826,20180319,Malfunction,U,N,...,None,None,None,None,None,None,None,None,None,None
3,10464153,2032227-2020-169500,N,Y,20200822,20200828,20190816,Malfunction,U,N,...,None,None,None,None,None,None,None,None,None,None
4,10526914,2032227-2020-176351,N,Y,20200905,20200914,20190409,Malfunction,U,N,...,None,None,None,None,None,None,None,None,None,None
5,10527157,2032227-2020-176487,N,Y,20200906,20200914,None,Malfunction,None,None,...,None,None,None,None,None,None,None,None,None,None
6,10533246,2032227-2020-176747,N,Y,20200905,20200915,None,Malfunction,None,None,...,None,None,None,None,None,None,None,None,None,None
7,10533475,2032227-2020-176882,N,Y,20200902,20200915,20200408,Malfunction,U,Y,...,None,None,None,None,None,None,None,None,None,None
8,10539011,2032227-2020-177388,N,Y,20200901,20200916,20181018,Malfunction,U,N,...,None,None,None,None,None,None,None,None,None,None
9,10554359,1219913-2020-00259,N,Y,20200728,20200918,None,Malfunction,None,None,...,None,None,None,None,None,None,None,None,None,None


In [ ]:
analyze_null_values(maude_match_lf, common_cols)
analyze_null_values(rename_maude_lf, common_cols)
overview_col(maude_match_lf, 'brand', n_rows=100)
overview_col(rename_maude_lf, 'brand', n_rows=100)
overview_col(maude_lf, 'device_0_brand_name', n_rows=1000)


=== 결측치 분석 ===
전체 행 수: 13,642,636

udi_di                                       :  4,415,922개 ( 32.37%)
model_number                                 :  2,352,042개 ( 17.24%)
catalog_number                               :  1,356,569개 (  9.94%)
brand                                        :    255,371개 (  1.87%)
manufacturer                                 :     54,479개 (  0.40%)

=== 결측치 분석 ===
전체 행 수: 13,642,636

udi_di                                       :  4,415,922개 ( 32.37%)
model_number                                 :  3,959,343개 ( 29.02%)
catalog_number                               :  1,528,916개 ( 11.21%)
brand                                        :    258,650개 (  1.90%)
manufacturer                                 :     85,122개 (  0.62%)
brand의 고유 개수: 144448


,head_brand,tail_brand
0,None,ZRA
1,0,ZRCEM
2,0 36IN PROLENE SUTURE,ZS3 DIAGNOSTIC ULTRASOUND SYSTEM
3,0 DEG RIGHT CUT GUIDE,ZSGM CUTTER 11 RATIO CAUTION SHARP
4,0 DEGREE DISTAL CUT BLOCK,ZSGM CUTTER 11 RATIO CAUTION SHARP
...,...,...
95,07 X 150MM GUIDE WIRE DOUBLE TROCAR,ZYSTON STRAIGHT 8H X 25LX10W LORDOTIC SPACER
96,070 ACCESS SYSTEM HIPOINT 70,ZYSTON STRAIGHT INLINE INSERTER
97,072 ASPIRATION SYSTEM,ZYSTON STRAIGHT INTERBODY FUSION SYSTEM
98,076 DRILL BIT MINI QC WITH 14 STOP445,ZYSTON STRUT


brand의 고유 개수: 186551


,head_brand,tail_brand
0,None,ZT TRIAL SLEEVE 16F XXL
1,0,ZT TRIAL SLEEVE 18B LRG
2,0 36IN PROLENE SUTURE,ZT TRIAL SLEEVE 18B SML
3,0 DEG RIGHT CUT GUIDE,ZT TRIAL SLEEVE 18D LRG
4,0 DEGREE DISTAL CUT BLOCK,ZT TRIAL SLEEVE 18D SML
...,...,...
95,040309U BRITEPRO SOLO MINI SINGLEUSE FIBER OPT...,ZYSTON STRAIGHT 13H X 20LX11W LORDOTIC SPACER
96,040341U BRITEPRO SOLO SINGLEUSE FIBER OPTIC HA...,ZYSTON STRAIGHT 8H X 25LX10W LORDOTIC SPACER
97,04041906504397,ZYSTON STRAIGHT INLINE INSERTER
98,043MCI 1251 BRACHYSOURCE,ZYSTON STRUT STANDARD INSERTER INNER SHAFT


device_0_brand_name의 고유 개수: 207710


,head_device_0_brand_name,tail_device_0_brand_name
0,None,ZEPHYR ENDOBRONCHIAL VALVE (EBV)
1,!!! POWERFLEXX,ZEPHYR ENDOBRONCHIAL VALVE SYSTEM
2,!!35X PROFLEXX,ZEPHYR SR
3,!!93-P PROFLEXX,ZEPHYR XL DR
4,!M1,ZEPHYR XL SR
...,...,...
995,"1.4MM X 12.8MM FRTR,CRNP","Ø8X 30MM BC IF SCRW, VENTED"
996,"1.4MM X 12.8MM FRTR,S",Ø9MM TI ADOLESCENT LAT ENTRY FEMORAL NAIL-EX/3...
997,1.5 3/3 HOLE 100 LEFT XLONG L,"Ø9X 20MM BC IF SCRW, VENTED"
998,1.5 3/3 HOLE 100* LEFT XLONG L,"Ø9X 30MM BC IF SCRW, VENTED"


### 결측 패턴 분석

In [ ]:
import polars as pl

# 결측 패턴 분석 (Polars LazyFrame 버전)
missing_pattern = {
    'overall_rate': (
        maude_match_lf
        .select(pl.col('udi_di').is_null().mean())
        .collect()
        .item()
    ),
    
    'by_manufacturer': (
        maude_match_lf
        .group_by('manufacturer')
        .agg(pl.col('udi_di').is_null().mean().alias('missing_rate'))
        .collect()
    ),
    
    'by_year': (
        maude_match_lf
        .with_columns(pl.col('date_received').str.to_datetime(format='%Y%m%d').dt.year().alias('year'))
        .group_by('year')
        .agg(pl.col('udi_di').is_null().mean().alias('missing_rate'))
        .collect()
    ),
    
    'by_product_code': (
        maude_match_lf
        .group_by('product_code')
        .agg(pl.col('udi_di').is_null().mean().alias('missing_rate'))
        .collect()
    )
}

InvalidOperationError: `year` operation not supported for dtype `str`

In [38]:
zero_brand = rename_maude_lf.filter(
    pl.col('brand').eq('00421871')
).head(10).collect()

zero_brand.to_pandas().transpose()

,0
mdr_report_key,22917392
report_number,3019004087-2025-01607
adverse_event_flag,N
product_problem_flag,Y
date_of_event,20250731
...,...
mdr_text_7_text_type_code,None
mdr_text_8_text,None
mdr_text_8_text_type_code,None
mdr_text_9_text,None


## MAUDE 데이터의 UDI를 깔끔하게

In [39]:
with open('primary.txt', 'w', encoding='utf-8') as f:
    pretty_primary = pformat(unique_udi_di.maude_primary, indent=4, width=80)
    f.write(pretty_primary)
    

with open('secondary.txt', 'w', encoding='utf-8') as f:
    pretty_secondary = pformat(unique_udi_di.maude_secondary, indent=4, width=80)
    f.write(pretty_secondary)

with open('notfound.txt', 'w', encoding='utf-8') as f:
    pretty_angry = pformat(unique_udi_di.angry, indent=4, width=80)
    f.write(pretty_angry)

In [40]:
group_cols = ['manufacturer', 'brand','model_number', 'lot_number']

In [41]:
group_lf = test.select(['udi_di', *group_cols]).group_by(group_cols).agg(
    pl.col('udi_di').n_unique().alias('udi_di_unique'),
)

outlier = group_lf.filter(
    pl.col('udi_di_unique').gt(1)
).select(pl.len()).collect().item()

print(f'UDI-DI 이상치는 {outlier}개 있습니다.')
group_lf.collect().drop_nulls().sort('udi_di_unique', descending=True).head(100).to_pandas()

UDI-DI 이상치는 60081개 있습니다.


,manufacturer,brand,model_number,lot_number,udi_di_unique
0,STAAR SURGICAL,IMPLANTABLE COLLAMER LENS ICL,VTICMO12.6,N/A,54
1,STAAR SURGICAL,IMPLANTABLE COLLAMER LENS ICL,VTICMO12.6,NA,44
2,STAAR SURGICAL,IMPLANTABLE COLLAMER LENS ICL,VTICMO13.2,N/A,40
3,STAAR SURGICAL,IMPLANTABLE COLLAMER LENS ICL,VTICMO 12.6,N/A,34
4,STAAR SURGICAL,IMPLANTABLE COLLAMER LENS ICL,VTICMO13.2,NA,33
...,...,...,...,...,...
95,ALCON LABORATORIES,ACRYSOF,SA60AT,ASKU,8
96,INSTITUT STRAUMANN,ITI DENTAL IMPLANT SYSTEM,SLA Titanium TL,1043,8
97,INSTITUT STRAUMANN,MODIFICATION TO ITI DENTAL IMPLANT SYSTEM,SLA Titanium TL,1013,8
98,INSTITUT STRAUMANN,MODIFICATION TO ITI DENTAL IMPLANT SYSTEM,SLA Titanium TL,1007,8


In [42]:
rename_maude_lf.select(group_cols + ['udi_di']).filter(
    pl.col('manufacturer').eq('DEXCOM'),
    pl.col('model_number').eq('9500-161'),
    pl.col('brand').eq('DEXCOM G7 CONTINUOUS GLUCOSE MONITORING SYSTEM'),
).drop_nulls().head(1000).collect().to_pandas()

,manufacturer,brand,model_number,lot_number,udi_di
0,DEXCOM,DEXCOM G7 CONTINUOUS GLUCOSE MONITORING SYSTEM,9500-161,1724060002,00386270003935
1,DEXCOM,DEXCOM G7 CONTINUOUS GLUCOSE MONITORING SYSTEM,9500-161,1724113001,00386270003935
2,DEXCOM,DEXCOM G7 CONTINUOUS GLUCOSE MONITORING SYSTEM,9500-161,1724118002,00386270003935
3,DEXCOM,DEXCOM G7 CONTINUOUS GLUCOSE MONITORING SYSTEM,9500-161,1724248006,00386270003935
4,DEXCOM,DEXCOM G7 CONTINUOUS GLUCOSE MONITORING SYSTEM,9500-161,1824133004,00386270003430
...,...,...,...,...,...
995,DEXCOM,DEXCOM G7 CONTINUOUS GLUCOSE MONITORING SYSTEM,9500-161,1724180004,00386270003935
996,DEXCOM,DEXCOM G7 CONTINUOUS GLUCOSE MONITORING SYSTEM,9500-161,1723317001,00386270004109
997,DEXCOM,DEXCOM G7 CONTINUOUS GLUCOSE MONITORING SYSTEM,9500-161,1724028001,00386270003423
998,DEXCOM,DEXCOM G7 CONTINUOUS GLUCOSE MONITORING SYSTEM,9500-161,1724114001,00386270003614
